<a href="https://colab.research.google.com/github/alexlinapp/python_tools_practice/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install fsspec==2023.6.0
!pip install --upgrade transformers tokenizers
# !pip uninstall -y torch torchtext
# !pip install torch==2.0.1+cu117 torchtext==0.15.2 -f https://download.pytorch.org/whl/torch_stable.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.6.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64

In [ ]:
# from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from transformers import GPT2Tokenizer
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.utils.data import TensorDataset
print(torch.__version__)

2.6.0+cu124


In [ ]:
from datasets import load_dataset

# This should load without issues
dataset = load_dataset("wikitext", "wikitext-2-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
inputs = tokenized['train']['input_ids']
print(type(tokenized['train']))

from itertools import chain

# Suppose 'tokenized' is your tokenized DatasetDict or Dataset

# Extract the list of token lists (input_ids) from your train split
input_ids_lists = tokenized['train']['input_ids']  # list of lists of ints

# Use itertools.chain to flatten efficiently into one big list
all_input_ids = list(chain.from_iterable(input_ids_lists))


<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
def group_strides_tensor(seq, bs):
  n = seq.size(0) // bs
  x = seq[:bs * n].reshape(bs, n, -1)


In [ ]:
input_tokens = torch.tensor(all_input_ids)[:200000]
bs = 64
seq_len = 5;
x = input_tokens.unfold(0, seq_len, 1)[:-1] # drops the last window
y = input_tokens[seq_len:]

windows = input_tokens.unfold(0, seq_len + 1, 1)[:-1]
x1 = windows[:, :-1]
y1 = windows[:, 1:]
tokendataset1 = TensorDataset(x1, y1)
tokendataset = TensorDataset(x, y)
dataloader = DataLoader(tokendataset, batch_size=bs, shuffle=True, drop_last=True)
dataLoader1 = DataLoader(tokendataset1, batch_size=bs, shuffle=True, drop_last=True)
first = next(iter(dataloader))
first[0].shape, first[1].shape
m1 = LMModel1(vocab_sz=tokenizer.vocab_size, n_hidden=16, seq_len = seq_len, n_layers=1)
m1(first[0])
# grouping by strides
#print(first[0].shape)
m2 = LSTMCell(16, 16, tokenizer.vocab_size, 5, bs, "cpu")
m2(first[0]).shape
#m2(first[0]).shape
first1 = next(iter(dataLoader1))
first1[0][:4], first1[1][:3]

(tensor([[ 1279,  2954,    29,   656,   262],
         [12566,   286,   262, 17757,   290],
         [25747,  3170,  1141, 27256,   764],
         [  632,  6774,   262,  9173,   736]]),
 tensor([[ 2954,    29,   656,   262,  5386],
         [  286,   262, 17757,   290,  8161],
         [ 3170,  1141, 27256,   764, 27182]]))

In [ ]:
class LMModel1(nn.Module):
  def __init__(self, vocab_sz, n_hidden, seq_len, n_layers=1, device="cpu"):
    super().__init__()
    self.i_h = nn.Embedding(vocab_sz, n_hidden) # Embedded layer. Maps inputs (vocab) to vector
    self.h_h = nn.Linear(n_hidden, n_hidden)    # Hidden layer
    self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
    self.h_o = nn.Linear(n_hidden, vocab_sz)    # Output layer
    self.seq_len = seq_len
    self.h = 0#torch.zeros(n_layers, bs, n_hidden).to("cuda")
    self.device = device
  def forward(self, x):

    for i in range(self.seq_len):
      self.h = self.h + self.i_h(x[:,i])
      #print(self.h.shape, x[:,i].shape)
      self.h = F.relu(self.h_h(self.h))
      out = self.h_o(self.h)
      #print(out.shape)
    self.h = self.h.detach()
    return out
    # res,h = self.rnn(self.i_h(x), self.h)
    # #print(res.shape)
    # self.h = h.detach()
    # #print(res.shape)
    # return self.h_o(res[:,-1,:])
  def reset(self): self.h = 0


class LMModel2(nn.Module):
  def __init__(self, vocab_sz, n_hidden, seq_len, n_layers=1, device="cpu"):
    super().__init__()
    self.i_h = nn.Embedding(vocab_sz, n_hidden) # Embedded layer. Maps inputs (vocab) to vector
    self.h_h = nn.Linear(n_hidden, n_hidden)    # Hidden layer
    self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
    self.h_o = nn.Linear(n_hidden, vocab_sz)    # Output layer
    self.seq_len = seq_len
    self.h = 0#torch.zeros(n_layers, bs, n_hidden).to("cuda")
    self.device = device
  def forward(self, x):
    outputs = torch.zeros(x.size(0), self.seq_len, self.h_o.out_features, device=self.device)
    for i in range(self.seq_len):
      self.h = self.h + self.i_h(x[:,i])
      #print(self.h.shape, x[:,i].shape)
      self.h = F.relu(self.h_h(self.h))
      out = self.h_o(self.h)
      #print(out.shape)
      outputs[:, i, :] = out
    self.h = self.h.detach()
    return outputs.permute(0, 2, 1)
    # res,h = self.rnn(self.i_h(x), self.h)
    # #print(res.shape)
    # self.h = h.detach()
    # #print(res.shape)
    # return self.h_o(res[:,-1,:])
  def reset(self): self.h = 0



class LSTMCell(nn.Module):
    def __init__(self, embed, nh, vocab_size, seq_len, bs, device):
        super().__init__()
        self.forget_gate = nn.Linear(embed + nh, nh)
        self.input_gate  = nn.Linear(embed + nh, nh)
        self.cell_gate   = nn.Linear(embed + nh, nh)
        self.output_gate = nn.Linear(embed + nh, nh)
        self.i_he = nn.Embedding(vocab_size, embed)
        self.h_o = nn.Linear(nh, vocab_size)
        self.seq_len = seq_len
        self.h = torch.zeros(bs, nh).to(device)
        self.c = torch.zeros(bs, nh).to(device)
        self.nh = nh
        self.bs = bs
        self.device = device
    def forward(self, x):
        self.h = self.h.detach()
        self.c = self.c.detach()
        # self.h = torch.zeros(self.bs, self.nh).to(self.device)
        # self.c = torch.zeros(self.bs, self.nh).to(self.device)
        for i in range(self.seq_len):
          input = self.i_he(x[:,i])
          temp_h = torch.cat([self.h, input], dim=1)
          #print("temph: ", temp_h.shape)
          forget = torch.sigmoid(self.forget_gate(temp_h))
          #print("forget: ", forget.shape)
          forget_c = self.c * forget
          #print("self.c: ", self.c.shape)
          #print("forget_c: ", forget_c.shape)
          inp = torch.sigmoid(self.input_gate(temp_h))
          cell = torch.tanh(self.cell_gate(temp_h))
          self.c = forget_c + inp * cell
          out = torch.sigmoid(self.output_gate(temp_h))
          self.h = out * torch.tanh(self.c)
          #print("self.h: ", self.h)
          #print("self.c: ", self.c)
        return self.h_o(self.h)

    # def __init__(self, ni, nh, vocab_size, seq_len, device):
    #     super().__init__()
    #     self.ih = nn.Linear(ni, 4 * nh)
    #     self.hh = nn.Linear(nh, 4 * nh)
    #     self.i_he = nn.Embedding(vocab_size, ni)
    #     self.h_o = nn.Linear(nh, vocab_size)
    #     self.seq_len = seq_len
    #     # self.h = torch.zeros(bs, nh).to(device)
    #     # self.c = torch.zeros(bs, nh).to(device)
    # def forward(self, input):
    #     for i in range(self.seq_len):
    #       input_step = self.i_he(input[:,i])
    #       gates = (self.ih(input_step) + self.hh(self.h)).chunk(4, 1)
    #       ingate, forgetgate, outgate = map(torch.sigmoid, gates[:3])
    #       cellgate = gates[3].tanh()
    #       self.c = (forgetgate * self.c) + (ingate * cellgate)
    #       self.h = outgate * self.c.tanh()
    #     self.h = self.h.detach()
    #     self.c = self.c.detach()
    #     return self.h_o(self.h)

In [ ]:
def train_model(model, dataloader, optimizer, loss_fnc, device, epochs=4):
  model.to(device)
  model.train()
  for epoch in range(epochs):
    total_loss = 0
    total_correct = 0
    total_tokens = 0
    print("entering")
    for xb, yb in dataloader:
      xb = xb.to(device)
      yb = yb.to(device)
      optimizer.zero_grad()
      out = model(xb)        # shape: (batch_size, sequence_length)
      #print(out.shape)
      loss = loss_fnc(out, yb)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
      total_correct += (out.argmax(dim=1) == yb).sum().item()
      total_tokens += xb.shape[0]
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_tokens
    print(f"Epoch {epoch+1}/{epochs} — Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

In [ ]:
xb, yb = next(iter(dataloader))
modeltest = LSTMCell(16, 32, tokenizer.vocab_size, seq_len, bs, "cpu")
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modeltest.parameters(), lr=0.001)
loss = loss_func(modeltest(xb), yb)
loss.backward()
optimizer.step()
# for param in modeltest.parameters():
#   print(param.grad)
print(loss)
for j in range(1):
  print(xb[:5])
  print(yb[:5])
  optimizer.zero_grad()
  out = modeltest(xb)
  loss = loss_func(out, yb)
  print(loss)
  loss.backward()
  optimizer.step()
i = 0


modeltest = LMModel2(vocab_sz=tokenizer.vocab_size, n_hidden=32, seq_len = seq_len, n_layers=4)

for xb, yb in dataLoader1:
  optimizer.zero_grad()
  out = modeltest(xb)
  loss = loss_func(out, yb)
  print(loss)
  loss.backward()
  optimizer.step()
  i += 1
  if i == 1000:
    break

tensor(10.8392, grad_fn=<NllLossBackward0>)
tensor([[  796,   569, 18354,  7496, 17740],
        [  569, 18354,  7496, 17740,  6711],
        [18354,  7496, 17740,  6711,   796],
        [ 7496, 17740,  6711,   796,   220],
        [17740,  6711,   796,   220,   198]])
tensor([6711,  796,  220,  198, 2311])
tensor(10.8309, grad_fn=<NllLossBackward0>)
tensor(10.8608, grad_fn=<NllLoss2DBackward0>)
tensor(10.9029, grad_fn=<NllLoss2DBackward0>)
tensor(10.9609, grad_fn=<NllLoss2DBackward0>)
tensor(10.8912, grad_fn=<NllLoss2DBackward0>)
tensor(10.8952, grad_fn=<NllLoss2DBackward0>)
tensor(10.9020, grad_fn=<NllLoss2DBackward0>)
tensor(10.8965, grad_fn=<NllLoss2DBackward0>)
tensor(10.8299, grad_fn=<NllLoss2DBackward0>)
tensor(10.9337, grad_fn=<NllLoss2DBackward0>)
tensor(10.9692, grad_fn=<NllLoss2DBackward0>)
tensor(10.8907, grad_fn=<NllLoss2DBackward0>)
tensor(10.9456, grad_fn=<NllLoss2DBackward0>)
tensor(10.8499, grad_fn=<NllLoss2DBackward0>)
tensor(10.8760, grad_fn=<NllLoss2DBackward0>)
ten

KeyboardInterrupt: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LMModel1(vocab_sz=tokenizer.vocab_size, n_hidden=32, seq_len = seq_len, n_layers=4, device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fnc = nn.CrossEntropyLoss()
print(device)
train_model(model, dataloader, optimizer, loss_fnc, device, epochs=4)

cuda
entering
Epoch 1/4 — Loss: 6.6838, Accuracy: 0.1473
entering
Epoch 2/4 — Loss: 5.8001, Accuracy: 0.1773
entering
Epoch 3/4 — Loss: 5.4283, Accuracy: 0.1971
entering
Epoch 4/4 — Loss: 5.1291, Accuracy: 0.2142


In [ ]:
model3 = LMModel2(vocab_sz=tokenizer.vocab_size, n_hidden=32, seq_len = seq_len, n_layers=4, device=device)
optimizer3 = torch.optim.Adam(model3.parameters(), lr=0.001)
train_model(model3, dataLoader1, optimizer3, loss_fnc, device, epochs=4)

entering
Epoch 1/4 — Loss: 6.1012, Accuracy: 0.8238
entering
Epoch 2/4 — Loss: 5.1752, Accuracy: 1.0623
entering
Epoch 3/4 — Loss: 4.7780, Accuracy: 1.1929
entering
Epoch 4/4 — Loss: 4.5508, Accuracy: 1.2826


In [ ]:
model2 = LSTMCell(32, 32, tokenizer.vocab_size, seq_len, bs, device)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)
loss_fnc = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_model(model2, dataloader, optimizer2, loss_fnc, device, epochs=4)

entering
Epoch 1/4 — Loss: 6.7505, Accuracy: 0.1440
entering
Epoch 2/4 — Loss: 5.9308, Accuracy: 0.1865
entering
Epoch 3/4 — Loss: 5.5658, Accuracy: 0.2031
entering
Epoch 4/4 — Loss: 5.2715, Accuracy: 0.2213


In [ ]:
import time
start = time.time()
for xb, yb in dataloader:
    pass
print(f"DataLoader iteration took {time.time()-start:.2f} seconds")

DataLoader iteration took 19.69 seconds
